This notebook should provide accurace result around 0.996. However, it is not possible to submit, because they require to turn off Internet. Without Internet, is not possible to download pretrained model. Dead-end...

In [ ]:
from fastai.vision.all import *
import pandas as pd
%matplotlib inline
set_seed(3865)

# Convert data to image files
At the beginning I want to convert dataset into the typical vision flow for fastai.

In [ ]:
# this need only for the image conversion
import numpy as np
import os
import cv2

In [ ]:
# function to convert csv file into PNG images inside of categorized file tree

inpSize = 28 # input image size
def makeTree( tpath, csvpath, withlabel=True ): # make file tree at tpath from csv dataframe
    tpd = pd.read_csv(csvpath)
    # tpd = tpd[:30] # short test
    os.makedirs(tpath, exist_ok=True) # make train or valid or test folder
    whiteImg = np.ones((28,28))*255
    
    for i in range(len(tpd)):
        img = np.array(tpd.iloc[i,1:]).reshape(28,28)
        img2 = whiteImg - img
        catpath = tpath
        if withlabel:
            categ = str(tpd.iloc[i].label)
            catpath = tpath + '/' + categ
            os.makedirs(catpath, exist_ok=True)
        cv2.imwrite(catpath + '/' + str(i) + '.png', img2)
    
    return

In [ ]:
makeTree('./Kannada-PNG/train', '../input/Kannada-MNIST/train.csv', withlabel=True)

In [ ]:
# test files are not labeled, 
# so we put them into upper folder, otherwise they will all be auto-labeled as 'test'
makeTree('./test', '../input/Kannada-MNIST/test.csv', withlabel=False)

In [ ]:
# test if we really built an image
timg = cv2.imread('./Kannada-PNG/train/8/18.png', cv2.IMREAD_GRAYSCALE)
plt.imshow(timg, cmap='gray')

# Now is fastai part

In [ ]:
import albumentations as Alb
class AlbTransform(Transform):
    def __init__(self, aug): self.aug = aug
    def encodes(self, img: PILImage):
        aug_img = self.aug(image=np.array(img))['image']
        return PILImage.create(aug_img)
    
def get_augs(): return Alb.Compose([
    # Alb.InvertImg(p=1.), # just because I like it white
    Alb.ShiftScaleRotate(rotate_limit=20, border_mode=0, value=(255,255,255) ),
    # Alb.RandomResizedCrop(28,28),
])

# super small image sizes may not work on all networks
# also resizing will make edges smoother
item_tfms = [Resize(56), AlbTransform(get_augs())] 
batch_tfms = Normalize.from_stats(*imagenet_stats) 

In [ ]:
# create dataloader from the folder structure
dls = ImageDataLoaders.from_folder( './Kannada-PNG', train='train', valid=None, valid_pct=0., 
    item_tfms=item_tfms, batch_tfms=batch_tfms, bs=256, shuffle=True )

In [ ]:
dls.vocab

In [ ]:
# uncomment to test data loaders
dls.train.show_batch(max_n=12)
# dls.valid.show_batch(max_n=12)
len(dls.train.items)
# dls.vocab

In [ ]:
# load pre-tarined model
# resnet18 works well even on 28 x 28, achieve 0.99 after 4 epochs
# densenet121 require bigger image, 56x56 will work 
learn = vision_learner(dls, resnet18, path='.', 
    loss_func=FocalLoss(),  
    metrics=[accuracy, F1Score(average='weighted')]  )

In [ ]:
# learn.fine_tune(10, freeze_epochs=1, cbs=[ShowGraphCallback()])
learn.fine_tune(10, freeze_epochs=1) # callback will fail without validation data, removed

In [ ]:
tfnames = get_image_files('./test')
tst_dl = dls.test_dl(tfnames, with_labels=False, shuffle=False)

In [ ]:
# uncomment to see if dataloader is working
tst_dl.show_batch(max_n=12)

In [ ]:
preds = learn.tta(dl=tst_dl, n=32, use_max=False)

In [ ]:
predss = learn.dls.vocab[np.argmax(preds[0], axis=1)] # convert to our classes from probabilities

In [ ]:
# predss = list(range(0,5000))

In [ ]:
submit_df = pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
submit_df.label = predss
submit_df.to_csv('submission.csv', header=True, index=False)

In [ ]:
submit_df

# Clean up image files
Otherwise it will burn time comitting.

In [ ]:
from shutil import rmtree
rmtree('./Kannada-PNG', ignore_errors=True)
rmtree('./test', ignore_errors=True)